<center> <h2> Desafío entregable </h2> </center>

## Descarga de datos desde APIs públicas
### Comisión 32845
* **Tutor:** Anahí García
* **Alumno:** Carlos Alarcón Villegas

### Se importan las librerías necesarias:

In [1]:
import requests
import pandas as pd

In [2]:
api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
tema_busqueda = "Derrame cerebral"

In [3]:
url = f"https://www.googleapis.com/youtube/v3/search?part=snippet&q={tema_busqueda}&type=video&key={api_key}"
respuesta = requests.get(url)

Se verifica que la respuesta de la solicitud sea exitosa, comprobando el código de estado de la respuesta. 
* Si es 200, la solicitud fue exitosa y se puede continuar con el procesamiento.

In [4]:
# Si la solicitud es exitosa, extraer los datos de los videos y almacenarlos en un DataFrame
if respuesta.status_code == 200:
    # Se extraen los datos de los videos de la rta. JSON utilizando la función response.json()["items"].
    videos = respuesta.json()["items"]
    
    #Se crea una lista vacía que se utilizará para almacenar los datos de los videos. 
    data = []
    
    for video in videos:
        video_data = {}
        video_data["video_id"] = video["id"]["videoId"]
        video_data["Título"] = video["snippet"]["title"]
        video_data["Descripción"] = video["snippet"]["description"]
        video_data["Nombre del canal"] = video["snippet"]["channelTitle"]
        
        # Se construye la URL de la solicitud de estadísticas de la API de YouTube para el canal actual.
        url_canal = f"https://www.googleapis.com/youtube/v3/channels?part=statistics&id={video['snippet']['channelId']}&key={api_key}"
        respuesta_canal = requests.get(url_canal)
        
        # Se verifica que la respuesta de la solicitud de estadísticas sea exitosa, comprobando 
        # el código de estado de la respuesta. 
        if respuesta_canal.status_code == 200:
            canal_data = respuesta_canal.json()["items"][0]["statistics"]
            video_data["Cantidad de suscriptores"] = canal_data.get("subscriberCount", 0)
        else:
            video_data["Cantidad de suscriptores"] = 0
        
        video_data["Fecha de publicación"] = video["snippet"]["publishTime"]
        
        # Se construye la URL de la solicitud de estadísticas de la API de YouTube para el video actual.
        estadisticas_url = f"https://www.googleapis.com/youtube/v3/videos?id={video_data['video_id']}&part=statistics&key={api_key}"
        respuesta_estadisticas = requests.get(estadisticas_url)
        
        # Se verifica que la respuesta de la solicitud de estadísticas sea exitosa, comprobando 
        # el código de estado de la respuesta. 
        if respuesta_estadisticas.status_code == 200:
            estadisticas_data = respuesta_estadisticas.json()["items"][0]["statistics"]
            video_data["Cantidad de visualizaciones"] = estadisticas_data.get("viewCount", 0)
            video_data["Cantidad de Me gustas"] = estadisticas_data.get("likeCount", 0)
            video_data["Cantidad de comentarios"] = estadisticas_data.get("commentCount", 0)
        else:
            video_data["Cantidad de visualizaciones"] = 0
            video_data["Cantidad de Me gustas"] = 0
            video_data["Cantidad de comentarios"] = 0
        
        data.append(video_data)
    
    # Se crea un DataFrame utilizando la lista "data"
    df = pd.DataFrame(data)

# Si la solicitud de búsqueda falla, se muestra un mensaje de error.
else:
    print("Error en la solicitud de búsqueda.")


**Validamos la carga de datos mediante la impresión de las primeras 5 filas**

In [5]:
df.head()

,video_id,Título,Descripción,Nombre del canal,Cantidad de suscriptores,Fecha de publicación,Cantidad de visualizaciones,Cantidad de Me gustas,Cantidad de comentarios
0,7DfW328HDMk,3 Síntomas y Causas de DERRAME cerebral,Causas y síntomas de DERRAME o hemorragia cere...,Alberto Sanagustín,1300000,2022-11-06T22:55:43Z,68539,2964,34
1,oxHkomQxF3A,¿Cómo y por qué ocurren los derrames cerebrales?,El doctor Juan Rivera explica cómo y por qué o...,Univision Noticias,7630000,2014-10-19T21:32:37Z,407631,3302,156
2,drGymPkfKrM,Síntomas y factores de riesgo en los derrames ...,El Dr. José Biller diserta sobre factores de r...,Univision Noticias,7630000,2014-10-19T22:00:41Z,426944,2784,131
3,Tb7yvkVerjc,Infarto o derrame cerebral,,Hospital Nacional,12900,2017-05-22T17:17:44Z,249589,2370,214
4,lBnT9oWaxk4,"DERRAME CEREBRAL 💥 Síntomas, Causas y Tratamie...",Las hemorragias cerebrales son conocidas popul...,Alberto Sanagustín,1300000,2022-01-19T22:34:53Z,165020,3181,206


**Dimensiones del DataFrame**

In [6]:
df.shape

(5, 9)

**Validamos las columnas y el tipo de formato de c/columna**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   video_id                     5 non-null      object
 1   Título                       5 non-null      object
 2   Descripción                  5 non-null      object
 3   Nombre del canal             5 non-null      object
 4   Cantidad de suscriptores     5 non-null      object
 5   Fecha de publicación         5 non-null      object
 6   Cantidad de visualizaciones  5 non-null      object
 7   Cantidad de Me gustas        5 non-null      object
 8   Cantidad de comentarios      5 non-null      object
dtypes: object(9)
memory usage: 488.0+ bytes
